ResNet code is from https://github.com/kuangliu/pytorch-cifar

In [ ]:
!git clone https://github.com/edomerli/nesterov-weight-decay.git

!mv /kaggle/working/nesterov-weight-decay/* /kaggle/working/.

In [2]:
import os
import sys
import time
import math
import numpy as np
import random
import wandb
from argparse import Namespace

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torch.backends.cudnn as cudnn
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

from adam_nesterov_wd import AdamNesterovWD
from utils import seed_everything, progress_bar

ImportError: cannot import name 'TypeIs' from 'typing_extensions' (d:\Users\edo\envs\cv\Lib\site-packages\typing_extensions.py)

In [5]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])


def ResNet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])


def ResNet50():
    return ResNet(Bottleneck, [3, 4, 6, 3])


def ResNet101():
    return ResNet(Bottleneck, [3, 4, 23, 3])


def ResNet152():
    return ResNet(Bottleneck, [3, 8, 36, 3])

In [6]:
# Config
CONFIG = {
    "seed": 42,
    "adamw": "custom",  # either "custom" or "torch"
}

config = Namespace(**CONFIG)

# Wandb
wandb.login(key="14a7d0e7554bbddd13ca1a8d45472f7a95e73ca4")
wandb.init(project="nesterov-weight-decay", name=f"original_torch{torch.__version__}", config=CONFIG, sync_tensorboard=True)

wandb.define_metric("train/batch")
wandb.define_metric("train/loss", step_metric="train/batch")
wandb.define_metric("train/accuracy", step_metric="train/batch")

wandb.define_metric("test/epoch")
wandb.define_metric("test/loss", step_metric="test/epoch")
wandb.define_metric("test/accuracy", step_metric="test/epoch")
wandb.define_metric("test/best_accuracy", step_metric="test/epoch")

device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
NUM_EPOCHS = 200

# Seed
print('==> Seeding..')
seed_everything(config.seed)

# Data
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

# Model
print('==> Building model..')
net = ResNet18()
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)    

criterion = nn.CrossEntropyLoss()
if config.adamw == "custom":
    optimizer = AdamNesterovWD(net.parameters(), lr=0.1, weight_decay=5e-4, nesterov=True)
elif config.adamw == "torch":
    optimizer = optim.AdamW(net.parameters(), lr=0.1, weight_decay=5e-4)
else:
    raise ValueError(f"Unknown AdamW implementation: {config.adamw}. Choose either 'custom' or 'torch'.")
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=NUM_EPOCHS)


# Training
def train(epoch):
    print(f'\nEpoch: {epoch}')
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss_batch = loss.item()
        train_loss += train_loss_batch
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct_batch = predicted.eq(targets).sum().item()
        correct += correct_batch

        # progress_bar(batch_idx, len(trainloader), f'Loss: {train_loss/(batch_idx+1):.3f} | Acc: {100.*correct/total:.3f}% ({correct}/{total})')

        wandb.log({
            "train/batch": epoch*len(trainloader) + batch_idx,
            "train/loss": train_loss_batch,
            "train/accuracy": 100.*correct_batch/targets.size(0)
        })

    print(f"Training loss: {train_loss/len(trainloader):.3f} - training accuracy: {100.*correct/total:.3f}%")


def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

            # progress_bar(batch_idx, len(testloader), f'Loss: {test_loss/(batch_idx+1):.3f} | Acc: {100.*correct/total:.3f}% ({correct}/{total})')

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.pth')
        best_acc = acc

    wandb.log({
        "test/epoch": epoch,
        "test/loss": test_loss/len(testloader),
        "test/accuracy": 100.*correct/total,
        "test/best_accuracy": best_acc
    })

    print(f"Test loss: {test_loss/len(testloader):.3f} - Test accuracy: {100.*correct/total:.3f}% - Best test accuracy: {best_acc:.3f}%")


for epoch in range(start_epoch, start_epoch+NUM_EPOCHS):
    train(epoch)
    test(epoch)
    scheduler.step()

==> Seeding..
==> Preparing data..


100%|██████████| 170498071/170498071 [00:09<00:00, 18757991.55it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified
==> Building model..

Epoch: 0


KeyboardInterrupt: 